In [3]:
import pickle
import json
from collections import Counter
files = "./data/baidu/duie_train.json"
with open(files,"rb") as f:
    data = f.readlines()
index = 0
a = ['上映时间', '专业代码', '改编自', '修业年限', '成立日期', 'ondate_date', '董事长', '注册资本', '占地面积', '人口数量', '所属专辑', 'inwork_影视作品', '编剧', '歌手', '票房', '导演', '主演', 'period_number', '邮政编码', '作者', '简称', '面积', '作曲', '作词']
predicates = set()
max_leng = 0
len_ = []
for line in data:
    line = json.loads(line)
    spo_list = line['spo_list']
    text = line['text']
    max_leng = max(len(text),max_leng)
    len_.append(len(text))
    for spo in spo_list:
        s = spo['subject']
        p = spo['object']['@value']
        predicate = spo['predicate']
        if ( s.isdigit() or p.isdigit() ) and predicate == a[1]:
            # print(p+"\t"+predicate+"\t"+s+"\t"+text)
            predicates.add(predicate)
            index += 1
# print(predicates)
print(max_leng)
print(len(len_))
print(sum([1 if l>250 else 0 for l in len_]))


300
171293
246


In [ ]:
import numpy as np
import torch


aa = np.array([[0.0349,  0.0670, -0.0612, 0.0280, -0.0222,  0.0422],
         [-1.6719,  0.1242, -0.6488, 0.3313, -1.3965, -0.0682],
         [-1.3419,  0.4485, -0.6589, 0.1420, -0.3260, -0.4795]])
bb = np.array([[-0.0658, -0.1490, -0.1684, 0.7188,  0.3129, -0.1116],
         [-0.2098, -0.2980,  0.1126, 0.9666, -0.0178,  0.1222],
         [ 0.1179, -0.4622, -0.2112, 1.1151,  0.1846,  0.4283]])
cc = np.where(aa>0)     #合并a,b两个tensor，如果a中元素大于0
dd = np.where(bb>0)


In [ ]:
import torch
sub_heads = torch.tensor([2,3,2,5,2,7,6,7,8,2])
sub_tails = torch.tensor([5,7,5,2,7,3,8,7,8,7])
sub_head_mapping = torch.zeros(10,1,9)
sub_tail_mapping = torch.zeros(10,1,9)
sub_head_mapping.scatter_(-1,sub_heads.reshape(10,1,1),1)
sub_tail_mapping.scatter_(-1,sub_tails.reshape(10,1,1),1)

In [7]:
from transformers import BertTokenizer
text = "内地影视演员龚蓓苾携手周渝民、迪丽热巴、张彬彬、刘芮麟、赖艺、代斯、张赫等主创悉数出席"
tokenizer = BertTokenizer.from_pretrained("../chinese_bert_base")
# print(tokenizer.tokenize(text))
# token = ['龚', '蓓', '苾', '携', '手', '周', '渝', '民', '、', '迪', '丽', '热', '巴', '、', '张', '彬', '彬', '、', '刘', '芮', '麟', '、', '赖', '艺', '、', '代', '斯', '、', '张', '赫', '等', '主', '创', '悉', '数', '出', '席']
# tokenizer.convert_tokens_to_ids(token)

In [8]:
import torch
head_size = 50
heads=4
a = torch.rand(10,30,head_size* heads * 2)
input1 = torch.split(a,head_size*2,dim = -1)
print(input1[0].shape)
input2 = torch.stack(input1, axis=-2)
print(input2.shape)
qw, kw = input2[..., :head_size], input2[..., head_size:]
print(qw.shape)
print(qw[..., 1::2].shape)
qw2 = torch.stack([-qw[..., 1::2], qw[..., ::2]], 4)
print(qw2.shape)
qw2 = torch.reshape(qw2, qw.shape)
print(qw2.shape)

torch.Size([10, 30, 100])
torch.Size([10, 30, 4, 100])
torch.Size([10, 30, 4, 50])
torch.Size([10, 30, 4, 25])
torch.Size([10, 30, 4, 25, 2])
torch.Size([10, 30, 4, 50])


In [4]:
import tensorflow as tf
import tensorflow.keras.backend as K
inputs = tf.random_normal((10,20,head_size* heads * 2))
print(inputs.shape)

(10, 20, 400)


In [41]:
input1 = tf.split(inputs, heads, axis=-1)
print(len(input1))
print(input1[0].shape)
input2 = K.stack(input1, axis=-2)
print(input2.shape)
qw, kw = input2[..., :head_size], input2[..., head_size:]
print(qw.shape)
qw2 = K.stack([-qw[..., 1::2], qw[..., ::2]], 4)
print(qw2.shape)
qw2 = K.reshape(qw2, K.shape(qw))
print(qw2.shape)


4
(10, 20, 100)
(10, 20, 4, 100)
(10, 20, 4, 50)
(10, 20, 4, 25, 2)
(10, 20, 4, 50)


In [81]:
inputs = torch.rand((10,100,20))
output_dim = 50
position_ids = torch.arange(0, 100, dtype=torch.float).reshape(1,-1)
indices = torch.arange(0, output_dim // 2, dtype=torch.float)
indices = torch.pow(10000.0, -2 * indices / output_dim).reshape(1,-1)
embeddings = torch.matmul(position_ids.unsqueeze(-1),indices.unsqueeze(0))
embeddings = torch.stack([torch.sin(embeddings), torch.cos(embeddings)], axis=-1)
# embeddings = torch.reshape(embeddings, (-1, 100, output_dim))
print(embeddings.shape)
# embeddings = embeddings.repeat([10, 1, 1])
# torch.cat([inputs, embeddings],axis=-1).shape

torch.Size([1, 100, 25, 2])


In [76]:
inputs = tf.random_normal((10,100,20))
position_ids = K.arange(0, 100, dtype=K.floatx())[None]

indices = K.arange(0, output_dim // 2, dtype=K.floatx())
indices = K.pow(10000.0, -2 * indices / output_dim)
print(indices.shape)
embeddings = tf.einsum('bn,d->bnd', position_ids, indices)
embeddings = K.stack([K.sin(embeddings), K.cos(embeddings)], axis=-1)
embeddings = K.reshape(embeddings, (-1, 100, output_dim))

print(embeddings.shape)
embeddings = K.tile(embeddings, [10, 1, 1])
K.concatenate([inputs, embeddings])


(25,)
(1, 100, 50)


<tf.Tensor 'concat:0' shape=(10, 100, 70) dtype=float32>

In [9]:
qw = torch.rand(10,30,2,20)
kw = torch.rand(10,30,2,20)
# torch.matmul(qw.transpose(2,1),kw.permute(0,3,1,2))
print(qw.transpose(2,1).shape)
kww = kw.transpose(2,1)
print(kww.shape)
print(kww.transpose(3,2).shape)
print(kw.permute(0,2,3,1).shape)
kww.transpose(3,2) == kw.permute(0,2,3,1)

torch.Size([10, 2, 30, 20])
torch.Size([10, 2, 30, 20])
torch.Size([10, 2, 20, 30])
torch.Size([10, 2, 20, 30])


tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]],

         [[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]],


        [[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  .

In [6]:
x = tf.constant([[0., 1., 2.], [0., 1., 2.]])
y1 = tf.reduce_logsumexp(x)             # 3.1007533
y2 = tf.reduce_logsumexp(x, 0)

sess = tf.Session()
# Evaluate the tensor `c`.

print(sess.run(y1))
print(sess.run(y2))
x = torch.tensor([[0., 1., 2.], [0., 1., 2.]])
y2 = torch.logsumexp(x, dim=0)
print(y2)

3.1007533
[0.6931472 1.6931472 2.6931472]


In [34]:
import torch
from torch.nn.parameter import Parameter, UninitializedParameter
a= torch.zeros((3,20))
b= torch.zeros((3,20))
a[0,5] = 1
a[1,7] = 1
a[2,2] = 1
b[0,10] = 1
b[1,16]=1
b[2,14]=1
c = torch.rand(3,20,20)
print(c[0,5,10])
print(c[1,7,16])
print(c[2,2,14])

tensor(0.9429)
tensor(0.5409)
tensor(0.8270)


In [35]:
def th_gather_nd(x, coords):
    x = x.contiguous()

    inds = coords.mv(torch.LongTensor(x.stride()))
    x_gather = torch.index_select(x.contiguous().view(-1), 0, inds)
    return x_gather
head = torch.nonzero(a)[:,1]
tail = torch.nonzero(b)[:,1]
head_tail = torch.vstack((head,tail)).T
batch = c.shape[0]
index = torch.hstack([torch.arange(batch).reshape(-1,1),head_tail])
# torch.index_select(c,[1,2],head_tail)
# th_gather_nd(c,index)
index

tensor([[ 0,  5, 10],
        [ 1,  7, 16],
        [ 2,  2, 14]])

In [36]:
idx_chunked = head_tail.chunk(2,1)
masked = c[torch.arange(3).view(3,1),idx_chunked[0].squeeze(),idx_chunked[1].squeeze()]
# masked.expand(1,*masked.shape)
diag = torch.diag(masked)
diag


tensor([0.9429, 0.5409, 0.8270])

In [21]:
idx_chunked

(tensor([[5],
         [7]]),
 tensor([[10],
         [16]]))

In [14]:
torch.randint(0,2,(2,4,2)).chunk(2,2)

(tensor([[[1],
          [0],
          [1],
          [0]],
 
         [[1],
          [1],
          [1],
          [1]]]),
 tensor([[[0],
          [0],
          [1],
          [0]],
 
         [[0],
          [1],
          [1],
          [0]]]))

In [130]:
batch = index[:,0]
start = index[:,1]
end = index[:,2]
d= a+b
seq_len = d.shape[-1]
seqs = torch.zeros_like(d)
seqs[:,:]= torch.arange(seq_len)
# d[:,:start] = 1
start_index = (seqs<start.reshape(-1,1)).int()
end_index = (seqs<=end.reshape(-1,1)).int()

In [134]:
new_index = start_index+end_index
new_range = (new_index==1).int()
new_range

tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]],
       dtype=torch.int32)

In [135]:
    def get_range_head_tail(head,tail):
        head_index = torch.nonzero(head)[:,1]
        tail_index = torch.nonzero(tail)[:,1]
        mask = torch.zeros_like(head)
        seq_len = head.shape[-1]
        mask[:,:]= torch.arange(seq_len)
        start_range = (mask<head_index.reshape(-1,1)).int()
        end_range = (mask<=tail_index.reshape(-1,1)).int()
        new_mask = start_range+end_range
        new_mask = (new_mask==1).int()
        return new_mask
get_range_head_tail(a,b)

tensor([[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]],
       dtype=torch.int32)

In [ ]:
{"text": "1CN5A科技广场为2010年在淘宝成立的一家及数码产品,网络设备,音响设备,电脑周边产品，虚拟产品等于一体的综合性网络商店.", "spo_list": [{"predicate": "成立日期", "object_type": {"@value": "Date"}, "subject_type": "机构", "object": {"@value": "2010年"}, "subject": "CN5A科技广场"}, {"predicate": "成立日期", "object_type": {"@value": "Date"}, "subject_type": "机构", "object": {"@value": "2010年"}, "subject": "1CN5A科技广场"}]}


In [5]:
"CN5A科技广场" in "1CN5A科技广场"

True

In [ ]:
{(12, 15): [(36, 37, 28)], (12, 14): [(36, 38, 28)]}
[{'predicate': '朝代', 'object_type': {'@value': 'Text'}, 'subject_type': '历史人物', 'object': {'@value': '唐'}, 'subject': '王难得', 'h': {'name': '王难得', 'pos': [11, 14]}, 't': {'name': '唐', 'pos': [35, 36]}}, {'predicate': '朝代', 'object_type': {'@value': 'Text'}, 'subject_type': '历史人物', 'object': {'@value': '唐代'}, 'subject': '王难', 'h': {'name': '王难', 'pos': [11, 13]}, 't': {'name': '唐代', 'pos': [35, 37]}}]
'宝应二年(763年),王难得死于军中,十三岁的王氏因祖父的战功而入宫为唐代宗才人'

In [3]:
import numpy as np
text_len = 300
s2ro_map = {(2, 12): [(103, 109, 11), (58, 64, 9), (14, 17, 30), (35, 49, 26), (76, 93, 9), (65, 75, 9)], (2, 6): [(35, 39, 26)]}
joints = np.zeros((56,text_len,text_len))
for sub,tails in s2ro_map.items():
    subs = np.zeros((text_len,text_len))
    subs[sub[0]:sub[1],:] = 1
    tail = np.zeros((56,text_len,text_len))
    for ro in tails:
        tail[ro[2],:,ro[0]:ro[1]]=1
    joints += ((subs+tail)==2).astype("int")
if len(np.where(joints>=2)[0]):
    print(s2ro_map)
joints = (joints>=1).astype("int")

{(2, 12): [(103, 109, 11), (58, 64, 9), (14, 17, 30), (35, 49, 26), (76, 93, 9), (65, 75, 9)], (2, 6): [(35, 39, 26)]}


In [11]:
def extract_bboxes(mask):
    triple_listes = {}
    for j in range(mask.shape[0]):
        m = mask[j,:,:]
        h = np.where(np.any(m,axis = 0))[0]
        v = np.where(np.any(m,axis = 1))[0]
        if h.shape[0]:
            object_head,object_tail = h[[0,-1]]
            subject_head,subject_tail = v[[0,-1]]
            sub = (subject_head,subject_tail+1)
            if sub not in triple_listes:
                triple_listes[sub]=[(object_head,object_tail+1,j)]
            else:
                triple_listes[sub].append((object_head,object_tail+1,j))

    return triple_listes
print(extract_bboxes(joints))

{(2, 12): [(58, 93, 9), (103, 109, 11), (35, 49, 26), (14, 17, 30)]}


In [8]:
from collections import defaultdict
def judge_(sub_obj,candidate):
    same = False
    can_sub,can_obj = candidate
    sam_index = -1
    for i,(sub,obj) in enumerate(sub_obj):
        if can_sub.lower() in sub.lower() and can_obj.lower() in obj.lower():
            same = True
            break
        elif sub.lower() in can_sub.lower() and obj.lower() in can_obj.lower():
            same = True
            sam_index = i
            break
        elif sub.lower() in can_sub.lower() and can_obj.lower() in obj.lower():
            same = True
            break
        elif can_sub.lower() in sub.lower() and obj.lower() in can_obj.lower():
            same = True
            break

    return same,sam_index

def judge_same_sub_obj(spos):
    sub_obj = []
    new_spos = []
    for spo in spos:
        sub = spo['subject']
        obj = spo['object']['@value']
        if len(sub_obj)==0:
            sub_obj.append((sub,obj))
            new_spos.append(spo)
        else:
            print((sub,obj))
            same,sam_index = judge_(sub_obj,(sub,obj))
            print(same,sam_index)
            if same:
                if sam_index !=-1:
                    sub_obj[sam_index] = (sub,obj)
                    new_spos[sam_index] = spo
            else:
                new_spos.append(spo)
                sub_obj.append((sub,obj))
            
    return new_spos

def filter_data(spo_lists):
    new_spo_lists = []
    same_predicate = defaultdict(list)
    for spo in spo_lists:
        predicate = spo['predicate']
        same_predicate[predicate].append(spo)
    
    for key,values in same_predicate.items():
        value = judge_same_sub_obj(values)
        new_spo_lists.extend(value)
    return new_spo_lists
spo_lists = [{'predicate': '成立日期', 'object_type': {'@value': 'Date'}, 'subject_type': '机构', 'object': {'@value': '2008年'}, 'subject': '泰国朱拉', 'h': {'name': '泰国朱拉', 'pos': [42, 46]}, 't': {'name': '2008年', 'pos': [53, 58]}}, {'predicate': '成立日期', 'object_type': {'@value': 'Date'}, 'subject_type': '机构', 'object': {'@value': '2008年'}, 'subject': '朱拉美容集团', 'h': {'name': '朱拉美容集团', 'pos': [44, 50]}, 't': {'name': '2008年', 'pos': [53, 58]}}]
filter_data(spo_lists)

('朱拉美容集团', '2008年')
False -1


[{'predicate': '成立日期',
  'object_type': {'@value': 'Date'},
  'subject_type': '机构',
  'object': {'@value': '2008年'},
  'subject': '泰国朱拉',
  'h': {'name': '泰国朱拉', 'pos': [42, 46]},
  't': {'name': '2008年', 'pos': [53, 58]}},
 {'predicate': '成立日期',
  'object_type': {'@value': 'Date'},
  'subject_type': '机构',
  'object': {'@value': '2008年'},
  'subject': '朱拉美容集团',
  'h': {'name': '朱拉美容集团', 'pos': [44, 50]},
  't': {'name': '2008年', 'pos': [53, 58]}}]

In [9]:
text = "泰国朱拉都没有听说过,亲,你落伍了撒,泰国朱拉是泰国朱拉美容保健集团旗下的系列产品,泰国朱拉美容集团成立于2008年,与国内各大知名网站合作,推出适合女性魅力产品-泰国朱拉,关爱女性健康与美丽,致力于成为高品质的女性服务品牌,主要成分是野葛根,适用乳房偏小、松弛、外扩、平胸、产后下垂、发育不良、减肥减了胸等乳房缺憾问题"
text[42:]

'泰国朱拉美容集团成立于2008年,与国内各大知名网站合作,推出适合女性魅力产品-泰国朱拉,关爱女性健康与美丽,致力于成为高品质的女性服务品牌,主要成分是野葛根,适用乳房偏小、松弛、外扩、平胸、产后下垂、发育不良、减肥减了胸等乳房缺憾问题'

In [2]:
print('常量 PI 的值近似为：%5.3f。' % 13443555.23)

常量 PI 的值近似为：13443555.230。


In [3]:
import torch
a = torch.rand(10,200)
a = a.unsqueeze(-1)
b = a.repeat(1,1,10)
c = a.expand(-1,-1,10)


tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],

        [[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]],